<img src = "https://images2.imgbox.com/c1/79/4H1V1tSO_o.png" width="1200">

# ÁRVORES DE DECISÃO
---

## Entendendo problemas lineares e não-lineares
---

Quando aprendemos, no começo do curso, sobre regressão linear vimos que esse modelo funciona muito bem quando temos uma certa linearidade dos nossos dados.

Por exemplo, suponha que você trabalha em um restaurante e quer estimar quanto de gorjeta os seus garçons vão receber com base na conta final do restaurante. Logo, quanto maior a conta do restaurante, maior a gorjeta.

<img src = "https://images2.imgbox.com/2a/65/R3YKFXEs_o.png" width="500">


In [ ]:
from scipy.stats import pearsonr 
corr = pearsonr(data1, data2)
print("Correlação de Pearson: %.3f", corr)

Correlação de Pearson: 0.887

Relembrando um pouco, correlação é um valor que varia entre −1 até 1, em que 1 é uma correlação positiva perfeita e -1 é uma correlação negativa perfeita. Geralmente, um valor acima de 0.50 está associado com uma alta correlação positiva.


Vamos treinar um modelo de regressão para esse caso :)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = train_test_split(data1.reshape(-1, 1), data2.reshape(-1, 1), test_size=0.1)
model = LinearRegression()
model.fit(X_train, y_train)
r2_score(y_test, model.predict(X_test))

0.7238562525812482

Conseguimos um modelo de regressão com o r² muito bom :) 

### Podemos até fazer um plot da reta em relação aos dados:

<img src = "https://images2.imgbox.com/03/a0/V26UD1DI_o.png" width="500">

Contudo, vamos supor que **aconteceu uma doideira no restaurante por um tempo** e a relação das variáveis mudaram um pouco, seguindo o gráfico abaixo:

<img src = "https://images2.imgbox.com/20/fe/PAxG3z7T_o.png" width="500">

Ou seja, a relação claramente deixou de ser claramente linear. Vamos computar a correlação de Pearson.

In [ ]:
corr, _ = pearsonr(data1, data2)
print("Correlação de Pearson: %.3f" % corr)

Correlação de Pearson: 0.738

A correlação ainda é forte, mas é claramente menor do que a que calculamos inicialmente. Vamos ver o comportamento da regressão nesse caso?

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data1.reshape(-1, 1), data2.reshape(-1, 1), test_size=0.1)
model = LinearRegression()
model.fit(X_train, y_train)
r2_score(y_test, model.predict(X_test))

0.44411006406707687

Ou, seja, claramente a regressão não foi muito boa. Vamos plotar a reta para esse caso:

<img src = "https://images2.imgbox.com/2d/b8/tsVA2qwp_o.png" width="500">

Como a **regressão linear** resulta em uma **reta**, será que conseguimos fazer melhor?


# Introduzindo árvores de decisão
---

**Árvores de Decisão** são modelos não lineares e que, portanto, conseguem encontrar padrões que possuem esse tipo de relação entre as variáveis. Mas como ela funciona?

A ideia é relativamente simples. Uma árvore de decisão é construída ao **repartir** os dados seguindo regras de decisão. Cada dado, uma vez repartido, resulta em um subgrupo, definido como nó. Por exemplo:

**Se o valor da conta for maior que 20 reais e menor do que 60, então o valor a gorjeta deve ser 1.**

## O algoritmo das árvores de decisão
---

Podemos definir um algoritmo de árvore de decisão seguindo o seguinte o código, inspirado no [link](https://mlcourse.ai/articles/topic3-dt-knn/#2.-Decision-Tree):

````
def monta_arvore(L):
 cria um nó t
 se o critério de parada é True:
 defina um modelo preditivo para t
 else:
 encontre o melhor split binário L = L_left + L_right
 t.left = monta_arvore(L_left)
 t.right = monta_arvore(L_right)
 return t
````


Mas como que essas regras são definidas?

## Regression Trees
---

No caso de árvores de decisão aplicadas a problemas de regressão, o modelo busca **partir os dados** seguindo um critério de quebra. No caso, queremos agrupar os nossos dados de forma que esses dados resultem em um **grupo que minimizem o erro quadrático médio**. Relembrando a métrica, MSE, é definida por:

<img src = "https://images2.imgbox.com/87/9d/ADF1ELQf_o.png" width="300">

**Ou seja, queremos minimizar a média dos erros elevado ao quadrado!**

Vamos ao código para ilustrar isso:

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(criterion="mse", max_depth=4, min_samples_leaf = 20)
model.fit(X_train, y_train)
r2_score(y_test, model.predict(X_test))

0.5566920750334438

Fazendo o plot da “reta”:

<img src = "https://images2.imgbox.com/4f/8a/0rbqBFGJ_o.png" width="500">

Ou seja, **claramente** o valor aprendido **não segue uma reta**, mas sim regras mais “secas” de corte dos nossos dados.

````
import cv2
from pydotplus.graphviz import graph_from_dot_data
from sklearn.tree import export_graphvizdot_data = export_graphviz(
    model, filled=True, rounded=True,
    out_file=None
)
graph = graph_from_dot_data(dot_data) 
graph.write_png('tree.png')img = cv2.imread('tree.png')
plt.figure(figsize = (20, 20))
_ = plt.imshow(img)
````

<img src = "https://images2.imgbox.com/dc/3c/g3kpBFTi_o.png" width="900">

Nós podemos interpretar o valor acima da seguinte forma:

- Se o valor do preço da conta for **menor** que 21.98, **maior** que 15.96, **então** o preço da gorjeta, baseado na média de 39 amostras, deve ser 0.88

- Se o valor do preço da conta for **maior** que 21.98, **maior** que 60.0 e **menor** que 79.6, **menor do que** 65.18, **então** o preço da gorjeta, baseado na média de 21 amostras, deve ser 1.646

## Desafios de árvores de decisão
---

Claramente, poderíamos construir uma árvore de decisão em que os **nós** possuissem **apenas** uma **única amostra**. Contudo, isso claramente resultaria em árvore overfitada e esse é o maior problema das árvores de decisão: **elas overfittam** muito fácil e por conta disso eu coloquei alguns **hiperparâmetros** quando instanciei as classe DecisionTreeRegressor. Além do critério da árvore, os principais hiperparâmetros das árvores de Decisão são:

- max_depth- a profundidade máxima da árvore (no nosso exemplo, usamos 3)

- max_features — o número **máximo** de features que o modelo olha para escolher a melhor regra de decisão

- min_samples_leaf — o número mínimo de amostras que queremos ter na árvore. No caso, usamos 20.

## Classification Trees
---


Como Árvores de Decisão funcionam no caso de problemas de classificação?

A primeira coisa a se pensar, contudo é que, como **estamos lidando com um problema de classificação**, não podemos ter MSE como critério de divisão. Para definirmos o critério ideal, vamos pegar o seguinte problema de classificação:

Suponha que você queira prever a cor de uma bola baseada na sua posição:

<img src = "https://images2.imgbox.com/8e/55/iyeWgkQi_o.png" width="800">

Nesse exemplo, temos 9 bolas azuis e 11 bolas amarelas. Se aleatoriamente pegarmos uma bola, ela será azul com a probabilidade 9/20 e amarela com probabilidade 11/20. Com isso, conseguimos ver que a chance de pegar uma bola de cada cor é mais ou menos a mesma. A partir dessa ideia, precisamos definir um conceito **novo**, chamado **entropia**.

## Entropia de Shannon
---

A entropia de Shannon é um conceito da **teoria da informação** que define o grau de **caos** de um sistema. Quanto maior a entropia, menos ordenado é um ambiente. Ela é uma métrica que é definida da seguinte forma:

<img src = "https://images2.imgbox.com/f8/24/pCVOko8T_o.png" width="300">

O resultado da entropia é medido em bits e pode ser um valor entre 0 e 1, em que 0 seria um sistema bem ordenado, com alto grau de certeza e 1, um alto grau de incerteza. O **log** aqui é na base do **número de classes que estamos usando**. Como no nosso caso, estamos lidando com 2 classes, o log é na base 2.

Graficamente, ela pode ser ilustrada pelo gráfico abaixo:

<img src = "https://images2.imgbox.com/be/2d/kajdRHJG_o.png" width="500">

Ou seja, segundo o exemplo, quando a probabilidade de pertencer a classe positva é 1 (apenas exemplos positivos), ou 0 (apenas exemplos negativos), a entropia resultante é 0, porquê **existe uma alta certeza da organização do ambiente**.

Disso, conseguimos derivar outra métrica, que é o ganho de informação:

𝐼𝐺(𝑌,𝑋)=𝐸(𝑋)−𝐸(𝑌|𝑋)

O ganho de informação de Y para X é dado pela Entropia de X menos a Entropia de Y dado X. **Na prática, quanto maior redução de incertezas, maior o ganho de informação possível**. Ou seja, existe uma relação **inversa** entre a entropia e o ganho de informação.

No caso, a entropia do sistema que estamos lidando é dado por:

<img src = "https://images2.imgbox.com/44/67/4UiIIbxa_o.png" width="500">

<img src = "https://images2.imgbox.com/e0/d9/I1lQ8qeC_o.png" width="800">

Logo, o nosso Y é basicamente dado pelo sistema pela regra de decisão Y ≤ 12. E X é o sistema todo, sem nenhuma regra de quebra até então (E(X)=H_0).

O grupo da esquerda tem 13 bolas, sendo 8 azuis e 5 amarelas, tendo como entropia H_1 e o grupo da direita tem 7 bolas, sendo 6 amarelas e 1 azul, tendo a entropia H_2.

<img src = "https://images2.imgbox.com/b9/5d/TytOxTQW_o.png" width="400">

Então, podemos calcular o ganho de informação dessa parte do nosso dataset da seguinte forma:

<img src = "https://images2.imgbox.com/92/e2/Uh4qDz1h_o.png" width="500">

Ou seja, o uso da regra de decisão 𝑋≤12 resulta em um sistema mais ordenado que o anterior, uma vez que existem algum ganho de informação.

Repetimos, então, o processo:

<img src = "https://images2.imgbox.com/75/85/7NHrbt1b_o.png" width="900">

Para o grupo da direita, a gente precisa apenas fazer uma partição a mais, uma vez que temos apenas uma bola na cor azul (basta perguntar se a posição da bola é menor ou igual que 18). Contudo, para o grupo da esquerda, precisamos repetir o processo três vezes. **Além disso, também é valido dizer que a entropia de um grupo onde todas as bolas são da mesma cor é 0.**

<img src = "https://images2.imgbox.com/af/5f/QdHYt2xs_o.png" width="500">

## Só podemos usar a entropia?
---

Não! Assim como na árvore para regressão, existem outros critérios de definição além da entropia, como o **Indice de Gini**. A idéia é que a relação entre as métricas seja parecida e isso pode ser visto com facilidade pelo gráfico abaixo:

<img src = "https://images2.imgbox.com/12/47/qmUMKxIh_o.png" width="500">

Em que o Gini e o missclassification são computadas, respectivamente por:

<img src = "https://images2.imgbox.com/c3/86/mZRTQIyg_o.png" width="500">

Na prática, as diferentes métricas (tendem a convertir para o mesmo resultado). A única diferença mais “prática” é que o índice de Gini é computacionalmente mais eficiente do que a entropia por não ter o cálculo do log. Além disso, se multiplicarmos o índice de Gini por 2, temos uma curva praticamente idêntica à entropia.

<img src = "https://images2.imgbox.com/cf/5e/NIlAsiFl_o.png" width="500">

## Aplicando um Exemplo
---

Suponha que queremos criar um modelo que separe corretamente as classes abaixo:

<img src = "https://images2.imgbox.com/8e/d6/TT0Ez7bh_o.png" width="500">

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf_tree = DecisionTreeClassifier(criterion='entropy', max_depth=3) # training the tree
clf_tree.fit(train_data, train_labels)

Podemos imprimir a **fronteira de decisão**

<img src = "https://images2.imgbox.com/78/a2/L8Qt23Bq_o.png" width="500">

In [ ]:
dot_data = export_graphviz(
    clf_tree, filled=True, rounded=True,
    out_file=None
)
graph = graph_from_dot_data(dot_data) 
graph.write_png('tree_classifier.png')
img = cv2.imread('tree_classifier.png')
plt.figure(figsize = (20, 20))
_ = plt.imshow(img)

<img src = "https://images2.imgbox.com/64/33/cis0NkrT_o.png" width="800">

## Um gostinho do que vem por aí
---

O interessante das Árvores de Decisão é que eles são a base para muitos algoritmos populares, como **Random Forests** e **Gradient Boosting**, que, por sua vez, são bastante utilizados na prática.

O interessante desses algoritmos é que eles são capazes de capturar não linearidades de uma maneira melhor. Pegando o exemplo da gorjeta, que tal compararmos uma árvore de decisão comum (o exemplo no começo do post) e uma random forest.

<img src = "https://images2.imgbox.com/84/25/fKteavyL_o.png" width="600">

<img src = "https://images2.imgbox.com/83/d6/dGUdHyvJ_o.png" width="500">


## Conclusão

Parabéns! Vocês acabaram de aprender um pouco mais sobre um dos modelos mais clássicos de machine learning, as **Decision Trees** :)

## Referências

https://towardsdatascience.com/https-medium-com-lorrli-classification-and-regression-analysis-with-decision-trees-c43cdbc58054

https://mlcourse.ai/articles/topic3-dt-knn/#2.-Decision-Tree

https://towardsdatascience.com/entropy-how-decision-trees-make-decisions-2946b9c18c8

https://scikit-learn.org/stable/modules/tree.html

---